# SEC Filing Analysis by FinMAS

This notebook can be used to run an analysis by SEC filing crew instead of using the UI / panel app.

In [ ]:
# Convenience function to change the working directory to the root of the project if necessary
from pathlib import Path
import os

if not Path("finmas").exists():
    os.chdir("..")
print(f"Current working directory: {Path.cwd()}")

/home/ivar/repos_ivar/agentic-llm-for-better-results


In [ ]:
# imports
import time

from edgar import Company, set_identity
from IPython.display import Markdown, display

from finmas.crews import SECFilingSectionsCrew
from finmas.crews.utils import CrewRunMetrics, save_crew_output
from finmas.utils.common import format_time_spent

set_identity("John Doe john.doe@example.com")

To be able to run this notebook make sure the necessary API_KEY for the LLM is set in the `.env` file
or set in the environment variables manually

| provider | API_KEY |
| --- | --- |
| groq | GROQ_API_KEY |
| openai | OPENAI_API_KEY |

## Configuration

In [ ]:
TICKER = "MSFT"
FILING_FORM = "10-K"

LLM_PROVIDER = "groq"
LLM_MODEL = "llama3-8b-8192"
EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"

SIMILARITY_TOP_K = 3
# LLM configuration
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 1024

## Get filing

In [ ]:
filing = Company(TICKER).get_filings(form=FILING_FORM).latest(1)

## Create the crew

In [ ]:
start = time.time()
crew = SECFilingSectionsCrew(
    ticker=TICKER,
    llm_provider=LLM_PROVIDER,
    llm_model=LLM_MODEL,
    embedding_model=EMBEDDING_MODEL,
    filing=filing,
    similarity_top_k=SIMILARITY_TOP_K,
    llm_temperature=LLM_TEMPERATURE,
    llm_max_tokens=LLM_MAX_TOKENS,
)

In [ ]:
index_creation_metrics_message = ""
for attr in dir(crew):
    if attr.endswith("index_creation_metrics"):
        index_creation_metrics_message += (
            f"{attr.replace('_', ' ').title()}:  \n{getattr(crew, attr).markdown()}\n\n"
        )
display(Markdown(index_creation_metrics_message))

## Run the analysis

In [ ]:
inputs = {"ticker": TICKER, "form": filing.form}
output = crew.crew().kickoff(inputs=inputs)
time_spent = time.time() - start
display(f"Time spent: {format_time_spent(time_spent)}")

## Save the results

In [ ]:
crew_run_metrics = CrewRunMetrics(
    config=crew.config, token_usage=output.token_usage, time_spent=time_spent
)
file_path = save_crew_output(crew_run_metrics, output.raw)
display(f"Output saved to: {file_path}")

## Display the results

In [ ]:
crew_output = file_path.read_text(encoding="utf-8")
display(Markdown(crew_output))